In [0]:
entity_name = dbutils.widgets.get("entity_name").capitalize()

In [0]:
if entity_name.endswith('incremental'):
    load_type = 'incremental'
else:
    load_type = 'full'

In [0]:
%run ./CommonUtils

In [0]:
%run ./CreateOrReplaceTables

In [0]:
catalog_name = "bronze_incremental_schema" if entity_name.endswith('_incremental') else "bronze_schema"
src_df = spark.read.table(f"{catalog_name}.{entity_name.split('_inc')[0]}").drop('processing_dttm')

In [0]:
if "lastUpdateDate" in src_df.columns:
    src_df = src_df.withColumnRenamed("lastUpdateDate","sourceLastUpdateDate")

In [0]:
if load_type == 'full':
    src_df = src_df.withColumn("processing_dttm",current_timestamp())
    crt_execute(entity_name,'silver',truncate_flag=False,src_df=src_df)

In [0]:
# df = spark.read.table(f"{get_catalog_name(entity_name,'bronze')}.{entity_name.split('_inc')[0]}")

In [0]:
# df = add_audit_columns(df,ingestion_layer="silver",entity_name=entity_name)

In [0]:
# table_exist_check = spark.sql(f"show tables in {get_catalog_name(entity_name,'silver')}").select('tableName').filter(f"tableName = '{entity_name.split('_inc')[0]}'").count() == 1
# if table_exist_check:
#     print('table exists')
# else:
#     dbutils.notebook.run('./Create Or Replace Tables',300,{"ingestion_layer":"silver","entity_name":entity_name})

In [0]:
# generate_clean_file(mountPoint,entity_name,df,ingestion_layer="silver")

In [0]:
# create_or_replace_table(mountPoint,entity_name, ingestion_layer="silver")

In [0]:
print("Silver Ingestion Completed")